In [9]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64


# Changed animal_shelter and AnimalShelter to match CRUD Python module file name and class name
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################

# Changed username and password and CRUD Python module name
username = "aacuser"
password = "luna123"
shelter = AnimalShelter(username, password)


# Class read method that supports return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))


#########################
# Dashboard Layout / View
#########################
                               
app = JupyterDash('SimpleExample')

# Added Grazioso Salvare’s logo
image_filename = 'Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    
    html.Div(
    html.Center(
        html.A(
            [html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'height':200,'width':200},
                alt="SNHU")],
                href='https://www.snhu.edu')
    )),
    
    html.Center(
        html.B(html.H2('SNHU CS-340 Dashboard - Kerrian Offermann', style={'color':'pink'}))),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='radio-id',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value':'reset'}
            ],
                labelStyle={'display':'inline'}
        ),
    ]),
        
#FIXED: Add in code for the interactive filtering options. For example, Radio buttons, drop down, 
        #checkboxes, etc.
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        fixed_columns={'headers':True, 'data':1},
        style_table={'minWidth':'100%'},
        style_header={
            'backgroundColor':'rgb(255,207,204)',
        },
        
        editable=True,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="True",
        row_selectable="single",
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
        
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    
    html.Div(className='row',
        style={'display':'flex'},
        children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
        ]),
    
])


#############################################
# Interaction Between Components / Controller
#############################################


# Generating columns
 
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])


def update_styles(selected_columns):
    return [{
        'if': {'column_id': i },'background_color': '#DBFEFF'} for i in selected_columns]    





# Generating radio buttons
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radio-id','value')])


def update_dashboard(value):  
    
# Create search criteria for water rescue dogs
    if value == 'water':
        df = pd.DataFrame(list(shelter.read_all(
            {
            "animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, 
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}
            })))
        
# Create search criteria for mountain and wilderness dogs                                     
    elif value == 'mountain':
        df = pd.DataFrame(list(shelter.read_all(
            {
            "animal_type":"Dog",
            "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky",
                           "Rottweiler"]}, 
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}
            })))

# Create search criteria for disaster rescue and individual tracking
    elif value == 'disaster':
        df = pd.DataFrame(list(shelter.read_all(
            {
            "animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound",
                           "Rottweiler"]}, 
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}
            })))

# Reset all filters
    else:
        df = pd.DataFrame(list(shelter.read_all({})))

# Return data and columns from database when filtering         
        
    columns = [{"name":i, "id":i, "deletable": False, "selectable":True} for i in df.columns]
    data = df.to_dict('records')  
    return (data,columns)





# Generating Pie Chart #

@app.callback(
    Output('graph-id',"children"),
    [Input('datatable-id',"derived_viewport_data")])


def update_graphs(viewData):       
    dff = pd.DataFrame.from_dict(viewData)
    column = dff.iloc[:,4]
    
    fig = px.pie(data_frame=dff, names=column, title = "Available Rescues by Breed",
                color_discrete_sequence=px.colors.sequential.RdBu)
    
    return [
        dcc.Graph(
            id='graph=id',
            figure=fig)
    ]
   


# Generating Geolocation Map #

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "selected_rows")])

def update_map(viewData,row):
    dff = pd.DataFrame.from_dict(viewData)

    return [
        dl.Map(style={'width':'500px', 'height':'500px'}, center=[30.75,-97.48], zoom=10, children=[
               dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[30.75,-97.48], children=[
                        dl.Tooltip(dff.iloc[0,4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
app